In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
	cluster_url=CLUSTER_URL,
	auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY,
        "X-Goog-Api-Key": GOOGLE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
	)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__
live = client.is_live()
connected = client.is_connected()

print(f"Weaviate Ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"Weaviate Live: {client.is_live()}")
print(f"Client Connected: {connected}")

In [ ]:
# Define your query vector (the same vector from your GraphQL query)
query_vector = [0.1, 0.2, 0.3, 0.4, 0.5]  # Replace with your actual vector
# Get the collection
collection = client.collections.get("MyCollection")

# Perform the near vector query
response = collection.query.near_vector(
    near_vector=query_vector,
    limit=1
)

# Print the results
for obj in response.objects:
    print(obj)

In [ ]:
# Use OpenAI to generate an embedding for a word
from openai import OpenAI

oa_client = OpenAI(api_key=OPENAI_API_KEY)

def embed(word: str):
    resp = oa_client.embeddings.create(model="text-embedding-3-small", input=word)
    return resp.data[0].embedding

vector = embed("")
print(f"Vector dimensionality: {len(vector)}")
print(vector)